In [46]:
import pandas as pd
import numpy as np
import time
import geopandas as gp
import os
import tables


In [2]:
pd.__version__

'0.24.1'

In [47]:
store = pd.HDFStore('model_data.h5')

In [48]:
store = pd.HDFStore('model_data.h5')
store.keys()

['/annual_household_relocation_rates',
 '/blocks',
 '/edges',
 '/edges_simple',
 '/households',
 '/jobs',
 '/nodes',
 '/nodes_simple',
 '/non_residential_units',
 '/nonres_rents',
 '/persons',
 '/places',
 '/region',
 '/residential_units',
 '/travel_data']

In [5]:
households = store['households']
# households.index.name = 'household_id'
# households_reset = households.reset_index()
# households_reset.head()

In [6]:
persons = store["persons"]
persons.index.name = 'person_id'
persons_reset = persons.reset_index()


In [7]:
res_units = store["residential_units"]
res_units_reset = res_units.reset_index()


In [8]:
jobs = store["jobs"]
jobs_reset = jobs.reset_index()


#### Household Variables

In [9]:
def sort_incomes(df):
    HHINC_LESS20K = sum(df["income"] < 20000)
    HHINC_20TO35K = sum((df["income"] > 20000) & (df["income"] <= 35000))
    HHINC_35TO50K = sum((df["income"] > 35000) & (df["income"] <= 50000))
    HHINC_50TO75k = sum((df["income"] > 50000) & (df["income"] <= 75000))
    HHINC_75TO100K = sum((df["income"] > 75000) & (df["income"] <= 100000))
    HHINC_100TO150K = sum((df["income"] > 100000) & (df["income"] <= 150000))
    HHINC_OVER150K = sum(df["income"] >= 150000)
    return [HHINC_LESS20K, HHINC_20TO35K,HHINC_35TO50K,HHINC_50TO75k, HHINC_75TO100K,
           HHINC_100TO150K, HHINC_OVER150K]

In [10]:
def num_vehicles(df):
    NVEH_0 = sum(df["cars"] == 0)
    NVEH1 = sum(df["cars"] == 1)
    NVEH2 = sum(df["cars"] == 2)
    NVEH_3 = sum(df["cars"] >= 3)
    return [NVEH_0, NVEH1, NVEH2, NVEH_3]

In [11]:
def hh_size(df):
    HHSIZE_1 = sum(df["persons"] == 1)
    HHSIZE_2 = sum(df["persons"] == 2)
    HHSIZE_3 = sum(df["persons"] == 3)
    HHSIZE_4 = sum(df["persons"] == 4)
    HHSIZE_5 = sum(df["persons"] == 5)
    HHSIZE_6 = sum(df["persons"] == 6)
    HHSIZE_7 = sum(df["persons"] == 7)
    return [HHSIZE_1,HHSIZE_2,HHSIZE_3,HHSIZE_4,HHSIZE_5,HHSIZE_6,HHSIZE_7]

In [14]:
# def hh_unit_type(df):
#     HUTYPE_SF = 
#     HUTYPE_2TO9UNITS = 
#     HUTYPE_10_UNITS = 
#     HUTYPE_OTHER = 

#### Build household variable dataframe

In [92]:
start = time.time()
df_lol = []
blk_id_list = []
for blk_id, blk_df in households_reset.head(100000).groupby("block_id"):
    blk_id_list.append(blk_id)
    row = []
#     display(blk_df) #create a list that is a row, populate that list and append it to a list set up before the 4 loop
    row.extend(sort_incomes(blk_df)) # returns list with counts of households in each income bin
    row.extend(hh_size(blk_df))
    row.extend(num_vehicles(blk_df))
    df_lol.append(row)

households_df = pd.DataFrame(df_lol, columns=["HHINC_LESS20K", "HHINC_20TO35K", "HHINC_35TO50K", 
        "HHINC_50TO75k", "HHINC_75TO100K", "HHINC_100TO150K", "HHINC_OVER150K", "HHSIZE_1", "HHSIZE_2",
        "HHSIZE_3", "HHSIZE_4", "HHSIZE_5", "HHSIZE_6", "HHSIZE_7",
        "NVEH_0", "NVEH1", "NVEH2", "NVEH_3"], index=blk_id_list)

end = time.time()



In [58]:
# elapsed = end- start
# print("time:", elapsed)
# households_df.head(30)


#### Person-level variables

In [64]:
def hhtype(df):
    owner = sum(df["tenure"] == 1) > 0 #tenure code 1 is owner, 2 is renter 
    spouse = sum(df["relate"] == 1) > 0 #if there is at least one person who is a husband or a wife
    HHT_OWN_MARRIED = sum(owner and spouse)
#     HHT_OWN_FAMILY_MALE
#     HHT_OWN_FAMILY_FEMALE
    HHT_OWN_NONFAMILY_SINGLE = sum((df["persons"] == 1) and (not spouse) & owner) # one person, no spouses, owner
    HHT_OWN_NONFAMILY_NOTALONE = sum((df["persons"] > 1) and (not spouse) & owner)# >one person, no spouses, owner
    HHT_RENT_MARRIED = sum(not owner and spouse)
#     HHT_RENT_FAMILY_MALE
#     HHT_RENT_FAMILY_FEMALE
    HHT_RENT_NONFAMILY_SINGLE  = sum(( not owner) and (df["persons"] == 1))# one person, at least one spouse, renter
    HHT_RENT_NONFAMILY_NOTALONE = sum(( not owner) and (df["persons"] > 1))# >one person, at least one spouse, renter
    return [HHT_OWN_MARRIED, HHT_RENT_MARRIED, HHT_OWN_NONFAMILY_NOTALONE, HHT_OWN_NONFAMILY_SINGLE, 
            HHT_RENT_NONFAMILY_SINGLE, HHT_RENT_NONFAMILY_NOTALONE]

In [30]:
def education(df):
    #ILF = in labor force, NILF = not in labor force. Determined by worker binary 0 = NILF, 1 = ILF
    #Employment status not determined at this time.
    EDUC_LESSHS_ILF = sum((df["edu"] <= 15) & (df["worker"] == 1))
    EDUC_LESSHS_NILF = sum((df["edu"] <= 15) & (df["worker"] == 0))
    
    #HS = high school diploma (16) or GED (17)
    EDUC_HS_ILF = sum(((df["edu"] == 16) | (df["edu"] == 17)) & (df["worker"] == 1))
    EDUC_HS_NILF = sum(((df["edu"] == 16) | (df["edu"] == 17)) & (df["worker"] == 0))
    
    #SOMECOLLEGE = less than 1 year of college, no degree (18) more than 1 yr, no degree (19)
    EDUC_SOMECOLLEGE_ILF = sum(((df["edu"] == 18) | (df["edu"] == 19)) & (df["worker"] == 1))
    EDUC_SOMECOLLEGE_NILF = sum(((df["edu"] == 18) | (df["edu"] == 19)) & (df["worker"] == 0))
    
    #COLLEGE = associates (20), bachelors (21), masters (22), professional (23), doctorate (24)
    EDUC_COLLEGE = sum(((df["edu"] >= 20) & (df["edu"] <= 24)) & (df["worker"] == 1))
    EDUC_COLLEGE_NILF  = sum(((df["edu"] >= 20) & (df["edu"] <= 24)) & (df["worker"] == 0))
    
    EDUC_UNDER25 = sum(df["age"] < 25)
    EDUC_65PLUS = sum(df["age"] >=65)
    return [EDUC_LESSHS_ILF, EDUC_LESSHS_NILF, EDUC_HS_ILF, EDUC_HS_NILF, EDUC_SOMECOLLEGE_ILF, EDUC_SOMECOLLEGE_NILF, 
    EDUC_COLLEGE, EDUC_COLLEGE_NILF, EDUC_UNDER25, EDUC_65PLUS]

In [31]:
def gender(df):
    SEX_MALE = len(df[df["sex"] == 1])
    SEX_FEMALE = len(df[df["sex"] == 2])
    return [SEX_MALE, SEX_FEMALE]

In [32]:
def age(df):
    AGE_under15 = sum(df["age"] < 15)  
    AGE_15to24 = sum((df["age"] <= 24) & (df["age"] > 15))
    AGE_25to34 = sum((df["age"] <= 34) & (df["age"] > 25))
    AGE_35to44 = sum(df["age"] <= 44 & (df["age"] > 35))
    AGE_45to54 = sum(df["age"] <= 54 & (df["age"] > 45))
    AGE_55to64 = sum(df["age"] <= 64 & (df["age"] > 55))
    AGE_65plus = sum(df["age"] >=65)
    return [AGE_under15, AGE_15to24, AGE_25to34, AGE_35to44, AGE_45to54, AGE_55to64, AGE_65plus]

In [33]:
#create a combined table of households and people to perform these calcs. Merging on household id, resulting table
#has the same number of rows as the persons table
persons_and_households = pd.merge(persons_reset, households_reset, how="inner", left_on="household_id", right_on="household_id")
persons_and_households.head()

,person_id,age,earning,edu,hours,household_id,member_id,race_id,relate,sex,...,persons,cars,income,race_of_head,age_of_head,workers,children,tenure,recent_mover,block_id
0,1,61,28000,21,40,169,1,1,0,2,...,1,1,28000,1,61,1,0,1,0,170310101001000
1,2,61,0,13,0,101,1,1,0,2,...,1,0,8270,1,61,0,0,2,0,170310101001003
2,3,63,0,22,0,125,1,1,0,2,...,1,1,7800,1,63,0,0,2,0,170310101001000
3,4,63,0,22,0,126,1,1,0,2,...,1,1,7800,1,63,0,0,2,0,170310101001003
4,5,63,0,22,0,127,1,1,0,2,...,1,1,7800,1,63,0,0,2,0,170310101001000


In [34]:
def race(df):
    RACE_WHITE = len(df[df["race_id"] == 1])
    RACE_BLACK = len(df[df["race_id"] == 2])
    RACE_INDIAN = len(df[df["race_id"] == 3])
    RACE_ASIAN = len(df[df["race_id"] == 6])
    RACE_OTHER = len(df[df["race_id"] == 8])
#     RACE_HISPANIC = len(df["race_id"] == ) I dont think there is information in the UrbanSim data about hispanic origin
    return [RACE_WHITE, RACE_BLACK, RACE_INDIAN, RACE_ASIAN, RACE_OTHER]

In [65]:
start = time.time()
df_lol_persons = []
blk_id_list_persons = []
for blk_id_p, blk_df_p in persons_and_households.head(50).groupby("block_id"):
    display(blk_df_p)
    blk_id_list_persons.append(blk_id_p)
    row = []
    row.extend(gender(blk_df_p))
    row.extend(race(blk_df_p))
    row.extend(age(blk_df_p))  
    row.extend(education(blk_df_p))
    row.extend(hhtype(blk_df_p))
    df_lol_persons.append(row)
    
persons_df = pd.DataFrame(df_lol_persons, columns=["AGE_under15", "AGE_15to24", "AGE_25to34", "AGE_35to44",
    "AGE_45to54","AGE_55to64","AGE_65plus", "SEX_MALE", "SEX_FEMALE", "RACE_WHITE",
             "RACE_BLACK", "RACE_INDIAN", "RACE_ASIAN", "RACE_OTHER", "EDUC_LESSHS_ILF", "EDUC_LESSHS_NILF",
    "EDUC_HS_ILF", "EDUC_HS_NILF", "EDUC_SOMECOLLEGE_ILF", "EDUC_SOMECOLLEGE_NILF", "EDUC_COLLEGE", "EDUC_COLLEGE_NILF",
    "EDUC_UNDER25", "EDUC_65PLUS", "HHT_OWN_MARRIED, HHT_RENT_MARRIED", "HHT_OWN_NONFAMILY_NOTALONE", 
                "HHT_OWN_NONFAMILY_SINGLE", "HHT_RENT_NONFAMILY_SINGLE", "HHT_RENT_NONFAMILY_NOTALONE"], index=blk_id_list_persons)
end = time.time()



,person_id,age,earning,edu,hours,household_id,member_id,race_id,relate,sex,...,persons,cars,income,race_of_head,age_of_head,workers,children,tenure,recent_mover,block_id
0,1,61,28000,21,40,169,1,1,0,2,...,1,1,28000,1,61,1,0,1,0,170310101001000
2,3,63,0,22,0,125,1,1,0,2,...,1,1,7800,1,63,0,0,2,0,170310101001000
4,5,63,0,22,0,127,1,1,0,2,...,1,1,7800,1,63,0,0,2,0,170310101001000
9,10,61,10800,18,20,165,1,1,0,2,...,1,1,10800,1,61,1,0,1,0,170310101001000
15,16,62,0,19,0,109,1,1,0,2,...,1,0,14160,1,62,0,0,2,1,170310101001000
17,18,63,14000,21,40,170,1,1,0,2,...,1,1,14000,1,63,1,0,1,0,170310101001000
28,29,63,0,16,0,124,1,2,0,1,...,1,1,19000,2,63,0,0,2,0,170310101001000
29,30,61,0,16,0,107,1,2,0,1,...,1,0,2400,2,61,0,0,2,1,170310101001000
30,31,63,9500,21,37,168,1,1,0,1,...,1,1,24520,1,63,1,0,1,0,170310101001000
32,33,61,135000,22,45,7,1,1,0,1,...,2,2,146800,1,61,2,0,1,0,170310101001000


TypeError: 'bool' object is not iterable

In [36]:
persons_df.head(30)

,AGE_under15,AGE_15to24,AGE_25to34,AGE_35to44,AGE_45to54,AGE_55to64,AGE_65plus,SEX_MALE,SEX_FEMALE,RACE_WHITE,...,EDUC_LESSHS_ILF,EDUC_LESSHS_NILF,EDUC_HS_ILF,EDUC_HS_NILF,EDUC_SOMECOLLEGE_ILF,EDUC_SOMECOLLEGE_NILF,EDUC_COLLEGE,EDUC_COLLEGE_NILF,EDUC_UNDER25,EDUC_65PLUS
170310101001000,46,50,54,32,0,5,0,7,9,17,...,5,9,6,6,14,5,42,9,17,0
170310101001001,18,27,19,18,0,2,0,6,7,11,...,0,10,2,5,5,4,18,1,14,0
170310101001002,16,18,18,15,0,0,0,3,4,9,...,3,3,0,2,6,2,14,4,7,0
170310101001003,116,122,94,131,1,4,1,23,27,67,...,10,33,20,19,35,14,91,16,52,0
170310101002000,1,1,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,2
170310101002001,0,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
170310101002002,0,2,2,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,2
170310101002005,1,2,3,0,0,0,0,0,0,0,...,0,0,1,0,1,1,0,0,0,1
170310101002006,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
170310101002008,1,1,2,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,2


#### Employment

In [22]:
def jobs_naics(df):
    AGRICULTURE = df[df["sector_id"] == 11]
    MINING = df[df["sector_id"] == 21]
    UTILITIES = df[df["sector_id"] == 22]
    CONSTRUCTION = df[df["sector_id"] == 23]
    MANUFACTURING = df[df["sector_id"] >= 31 & df["sector_id"] <= 33]
    WHOLESALE = df[df["sector_id"] == 42]
    RETAIL = df[df["sector_id"] == 44 or df["sector_id"] == 45]
    TRANSPORTATION = df[df["sector_id"] == 48 or df["sector_id"] == 49]
    INFORMATION = df[df["sector_id"] == 51]
    FINANCE = df[df["sector_id"] == 52]
    REALESTATE = df[df["sector_id"] == 53]
    PROFESSIONAL = df[df["sector_id"] == 54]
    EDUCATION = df[df["sector_id"] == 61]
    MEDICAL_SOCIALWORK = df[df["sector_id"] == 62]
    ENTERTAINMENT = df[df["sector_id"] == 71]
    SERVICES = df[df["sector_id"] == 81]
    ADMINISTRATION = df[df["sector_id"] == 92]
    return [AGRICULTURE, MINING, UTILITIES, CONSTRUCTION, MANUFACTURING, WHOLESALE, RETAIL, TRANSPORTATION,
        INFORMATION, FINANCE, REALESTATE, PROFESSIONAL, EDUCATION, MEDICAL_SOCIALWORK, ENTERTAINMENT, SERVICE, ADMINISTRATION]
#     MILITARY = df[df["sector_id"] == ]
#     UNEMPLOYED = df[df["sector_id"] == ]

In [19]:
df_lol_jobs = []
blk_ids_jobs = []
for blk_id_j, df in jobs_reset.groupby("block_id"):
    blk_ids_jobs.append(blk_id_j)
    display(df)
    row = []
    row.extend(gender(blk_df_p))
    persons_df = pd.DataFrame(df_lol_persons, columns=["SEX_MALE",


,job_id,block_id,sector_id,nru_id
46,47,170310101002005,52,2307602
47,48,170310101002005,62,2307609
48,49,170310101002005,62,7052584
49,50,170310101002005,62,7052595
50,51,170310101002005,62,2307603
51,52,170310101002005,62,7052583
52,53,170310101002005,62,7052591
53,54,170310101002005,62,4680096
54,55,170310101002005,62,7052582
55,56,170310101002005,62,2307608


In [151]:
persons_df.head(50)


,SEX_MALE,SEX_FEMALE,RACE_WHITE,RACE_BLACK,RACE_INDIAN,RACE_ASIAN,RACE_OTHER,AGE_under15,AGE_over40
170310101001000,46,50,54,32,0,5,0,96,96
170310101001001,18,27,19,18,0,2,0,45,45
170310101001002,16,18,18,15,0,0,0,34,34
170310101001003,116,122,94,131,1,4,1,238,238
170310101002000,1,1,2,0,0,0,0,2,2
170310101002001,0,1,1,0,0,0,0,1,1
170310101002002,0,2,2,0,0,0,0,2,2
170310101002005,1,2,3,0,0,0,0,3,3
170310101002006,0,1,1,0,0,0,0,1,1
170310101002008,1,1,2,0,0,0,0,2,2


#### Land Use

In [50]:
blk_grp_shpfile = gp.read_file("tl_2017_17_bg/tl_2017_17_bg.shp")
land_area = blk_grp_shpfile[["GEOID", "ALAND"]]
land_area["GEOID"] = land_area["GEOID"].astype(str)

land_area.head()

/home/amelia/enter/envs/polaris/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,GEOID,ALAND
0,170190008001,3498215
1,170190008002,8690578
2,170190106042,106818683
3,170312909001,164717
4,171419612003,729540


In [80]:
land_use = pd.read_csv('Chicago_block_LandUse.csv') #land use file is at the block level, we want the block group level

land_use["GEOID"] = land_use["GEOID"].astype(str)
land_use["GEOID"] = land_use["GEOID"].str[:12] #take the first 11 digits of the FIPS to get the block group
land_use.drop(['Unnamed: 0','Unnamed: 2', 'propAreaCheck'], axis=1, inplace=True)

land_use.head()

/home/amelia/enter/envs/polaris/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,GEOID,L1111,L1112,L1130,L1140,L1151,L1211,L1212,L1214,L1215,...,L4210,L4220,L4230,L4240,L5000,L6100,L6200,L6300,L6400,L9999
0,170070106013,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031400,0.0,0.0
1,170070106013,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001993,0.0,0.0
2,170310101001,0.000000,0.0,0.728775,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.271225,0.0,0.0
3,170310101001,0.691130,0.0,0.142452,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166418,0.0,0.0
4,170310101001,0.624356,0.0,0.193838,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.181805,0.0,0.0


In [81]:
land_use_area = pd.merge(land_use, land_area, how='inner', left_on='GEOID', right_on='GEOID')
land_use_area.head()

,GEOID,L1111,L1112,L1130,L1140,L1151,L1211,L1212,L1214,L1215,...,L4220,L4230,L4240,L5000,L6100,L6200,L6300,L6400,L9999,ALAND
0,170070106013,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.031400,0.0,0.0,42871946
1,170070106013,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.001993,0.0,0.0,42871946
2,170310101001,0.000000,0.0,0.728775,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.271225,0.0,0.0,62463
3,170310101001,0.691130,0.0,0.142452,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.166418,0.0,0.0,62463
4,170310101001,0.624356,0.0,0.193838,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.181805,0.0,0.0,62463


In [82]:
lu_types = ['L1111', 'L1112', 'L1130', 'L1140', 'L1151', 'L1211', 'L1212',
       'L1214', 'L1215', 'L1216', 'L1220', 'L1240', 'L1250', 'L1310', 'L1321',
       'L1322', 'L1330', 'L1340', 'L1350', 'L1360', 'L1370', 'L1380', 'L1410',
       'L1420', 'L1431', 'L1432', 'L1433', 'L1450', 'L1511', 'L1512', 'L1520',
       'L1530', 'L1540', 'L1550', 'L1561', 'L1562', 'L1563', 'L1564', 'L1565',
       'L1570', 'L2000', 'L3100', 'L3200', 'L3300', 'L3400', 'L3500', 'L4110',
       'L4120', 'L4130', 'L4140', 'L4210', 'L4220', 'L4230', 'L4240', 'L5000',
       'L6100', 'L6200', 'L6300', 'L6400', 'L9999']
#27880000 square feet in a square mile
for lu in lu_types:
    land_use_area[lu+"Area_ft2"] = land_use_area["ALAND"]*land_use_area[lu]*27880000
    land_use_area.drop(lu,axis=1, inplace=True)



In [83]:
land_use_area.groupby(["GEOID"]).sum()

,ALAND,L1111Area_ft2,L1112Area_ft2,L1130Area_ft2,L1140Area_ft2,L1151Area_ft2,L1211Area_ft2,L1212Area_ft2,L1214Area_ft2,L1215Area_ft2,...,L4210Area_ft2,L4220Area_ft2,L4230Area_ft2,L4240Area_ft2,L5000Area_ft2,L6100Area_ft2,L6200Area_ft2,L6300Area_ft2,L6400Area_ft2,L9999Area_ft2
GEOID,,,,,,,,,,,,,,,,,,,,,
170070106013,85743892,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.991430e+13,0.000000e+00,0.000000e+00
170310101001,249852,2.515769e+12,8.882502e+10,2.719359e+12,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.641921e+12,0.000000e+00,0.000000e+00
170310101002,2566410,0.000000e+00,5.474584e+11,3.271958e+13,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,6.388902e+11,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.044696e+13,0.000000e+00,0.000000e+00
170310101003,1741649,1.503455e+12,9.132879e+11,2.013175e+13,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,4.700044e+11,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,7.785991e+10,1.618345e+13,0.000000e+00,0.000000e+00
170310102011,1342908,4.466962e+12,9.779375e+11,1.841833e+13,0.000000e+00,1.222253e+11,0.0,0.000000e+00,0.000000e+00,4.367282e+11,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.301810e+13,0.000000e+00,0.000000e+00
170310102012,10632366,9.297901e+12,4.077675e+12,1.459943e+14,0.000000e+00,5.660099e+10,0.0,0.000000e+00,0.000000e+00,1.734977e+13,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.028279e+14,0.000000e+00,0.000000e+00
170310102013,1790592,8.938775e+12,9.128778e+10,1.750635e+13,0.000000e+00,5.616866e+09,0.0,0.000000e+00,0.000000e+00,9.743679e+11,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.788946e+13,0.000000e+00,0.000000e+00
170310102021,2576280,6.338585e+12,2.024673e+12,1.319445e+13,0.000000e+00,0.000000e+00,0.0,4.872297e+12,0.000000e+00,7.088816e+12,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.409689e+13,0.000000e+00,0.000000e+00
170310102022,1845107,1.130406e+13,1.626665e+12,1.559902e+13,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,1.503263e+12,...,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.575736e+13,0.000000e+00,0.000000e+00
